# Pipeline for High-z Radio Galaxies 12: Create lists (files) with radio AGN candidates

## Introduction

In this file, three models will be applied consecutively in order to predict  
the detection of Radio Galaxies (radio AGN) and their redshift.  

In principle, this pipeline should be applied to data in Stripe 82. But  
it can be used with any other suitable dataset.

In [1]:
%matplotlib inline
# Static plots
#%matplotlib ipympl
# Interactive plots
import numpy as np
import matplotlib as mpl
import matplotlib.cm as cm
from matplotlib import ticker
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.patheffects as mpe
import matplotlib.patches as mpatches
from matplotlib.ticker import ScalarFormatter
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from astropy.visualization import LogStretch, PowerStretch
from astropy.visualization.mpl_normalize import ImageNormalize
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import ConfusionMatrixDisplay
import sklearn.pipeline
import colorcet as cc
from pycaret import classification as pyc
from pycaret import regression as pyr
from joblib import dump, load
import pandas as pd
import mpl_scatter_density
import global_variables as gv
import global_functions as gf

In [2]:
mpl.rcdefaults()

---

## Reading data

Flags.

In [3]:
save_plot_flag      = False
save_preds_flag     = False
load_models_flag    = True
predict_only_hi_z   = False
use_zeroth_model    = False
use_second_z_model  = False  # z >= 3.6 (with SMOGN), or, if needed, z >= 4.0

In [4]:
used_area           = 'HETDEX'  # can be 'S82', 'HETDEX', 'COSMOS'

In [5]:
file_name_dict      = {'S82': gv.file_S82, 'HETDEX': gv.file_HETDEX, 'COSMOS': gv.file_COSMOS}
file_name           = file_name_dict[used_area]

In [6]:
feats_2_disc_S82    = ['objID', 'RA_MILLI', 'DEC_MILLI', 'W1mag', 'W2mag', 'num_imputed', 'radio_detect']
feats_2_disc_HETDEX = ['objID', 'RA_MILLI', 'DEC_MILLI', 'W1mag', 'W2mag', 'num_imputed', 'radio_detect']
feats_2_disc_COSMOS = ['objID', 'RA_MILLI', 'DEC_MILLI', 'W1mag', 'W2mag', 'num_imputed', 'radio_detect', ]

feats_2_disc        = {'S82': feats_2_disc_S82, 'HETDEX': feats_2_disc_HETDEX, 'COSMOS': feats_2_disc_COSMOS}
features_2_discard  = feats_2_disc[used_area]

In [7]:
full_catalog_df     = pd.read_hdf(gv.cat_path + file_name, key='df').drop(columns=features_2_discard)

In [8]:
if used_area == 'S82':
    full_catalog_df.loc[:, 'LOFAR_detect'] = full_catalog_df.loc[:, 'VLAS82_detect'].copy()
    full_catalog_df = full_catalog_df.drop(columns=['VLAS82_detect'])
if used_area == 'COSMOS':
    full_catalog_df.loc[:, 'LOFAR_detect'] = full_catalog_df.loc[:, 'COSMOSVLA3_detect'].copy()
    full_catalog_df = full_catalog_df.drop(columns=['COSMOSVLA3_detect'])

Create features with class and combined redshift.

In [9]:
full_catalog_df['class']            = full_catalog_df.loc[:, 'is_AGN'].copy()
filter_non_confirmed                = np.array(full_catalog_df.loc[:, 'is_AGN'] == 1) | np.array(full_catalog_df.loc[:, 'is_gal'] == 1)
full_catalog_df.loc[~filter_non_confirmed, 'class'] = np.nan
idx_non_Z                           = full_catalog_df.loc[:, 'Z'].where(full_catalog_df.loc[:, 'Z'] > 0).isna()
full_catalog_df.loc[idx_non_Z, 'Z'] = full_catalog_df.loc[:, 'Z'].mask(idx_non_Z, full_catalog_df.loc[idx_non_Z, 'zsp'])

Create column for detection as Radio AGN

In [10]:
full_catalog_df['radio_AGN']      = (np.array(full_catalog_df.loc[:, 'is_AGN'] == 1) &\
                                     np.array(full_catalog_df.loc[:, 'LOFAR_detect'] == 1)).astype(int)

Discard minor features.

In [11]:
full_catalog_df                     = full_catalog_df.drop(columns=['is_AGN', 'is_SDSS_QSO', 'is_SDSS_gal', 'is_gal', 'zsp'])

#### As we want to predict, only use sources that have not previous spectroscopic classification

In [12]:
print(f'The used data set (in {used_area}) has {len(full_catalog_df):,} sources.')
print(f'And {np.sum(filter_non_confirmed):,} have previous spectroscopic classification.')

The used data set (in HETDEX) has 15,136,878 sources.
And 118,734 have previous spectroscopic classification.


In [13]:
# full_catalog_df                     = full_catalog_df.loc[~filter_non_confirmed]

In [14]:
print(f'This pipeline will predict properties for {len(full_catalog_df):,} sources in {used_area}.')

This pipeline will predict properties for 15,136,878 sources in HETDEX.


---

### Load models

In [15]:
if load_models_flag:
    if use_zeroth_model:
        star_clf        = pyc.load_model(gv.models_path + gv.star_model)  # star/no-star model
        cal_star_clf    = load(gv.models_path + gv.cal_str_model)  # calibrated model
    AGN_gal_clf         = pyc.load_model(gv.models_path + gv.AGN_gal_model)  #
    cal_AGN_gal_clf     = load(gv.models_path + gv.cal_AGN_gal_model)  # calibrated model
    radio_det_clf       = pyc.load_model(gv.models_path + gv.radio_model)  # without predicted AGN
    cal_radio_det_clf   = load(gv.models_path + gv.cal_radio_model)  # calibrated model
    redshift_reg        = pyr.load_model(gv.models_path + gv.full_z_model)  # to use on full sample
    if use_second_z_model:
        redshift_reg_2  = pyr.load_model(gv.models_path + gv.high_z_model)  # sources with predicted z >= 3.6

Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded


In [16]:
full_catalog_df = pyc.predict_model(AGN_gal_clf, data=full_catalog_df, probability_threshold=gv.AGN_thresh, raw_score=True, round=10)
full_catalog_df = full_catalog_df.drop(columns=['Score_0'])
full_catalog_df = full_catalog_df.rename(columns={'Label': 'pred_class', 'Score_1': 'Score_AGN'})
full_catalog_df['Score_AGN'] = np.around(full_catalog_df.loc[:, 'Score_AGN'], decimals=7)
pred_probs_AGN  = cal_AGN_gal_clf.predict(full_catalog_df.loc[:, 'Score_AGN'])
full_catalog_df['Prob_AGN'] = pred_probs_AGN

In [17]:
full_catalog_df = pyc.predict_model(radio_det_clf, data=full_catalog_df, probability_threshold=gv.radio_thresh, raw_score=True, round=10)
full_catalog_df = full_catalog_df.drop(columns=['Score_0'])
full_catalog_df = full_catalog_df.rename(columns={'Label': 'pred_radio', 'Score_1': 'Score_radio'})
full_catalog_df['Score_radio'] = np.around(full_catalog_df.loc[:, 'Score_radio'], decimals=7)
pred_probs_rad  = cal_radio_det_clf.predict(full_catalog_df.loc[:, 'Score_radio'])
full_catalog_df['Prob_radio'] = pred_probs_rad

In [18]:
full_catalog_df = pyr.predict_model(redshift_reg, data=full_catalog_df, round=8)
full_catalog_df = full_catalog_df.rename(columns={'Label': 'pred_Z'})
full_catalog_df['pred_Z'] = np.around(full_catalog_df.loc[:, 'pred_Z'], decimals=3)

In [19]:
redshift_tol    = 0.0
if use_second_z_model:
    full_catalog_df = pyr.predict_model(redshift_reg_2, data=full_catalog_df, round=6)
    filter_pred_z   = full_catalog_df.loc[:, 'pred_Z'] >= (gv.high_z_limit + redshift_tol)
    full_catalog_df.loc[:, 'pred_Z'] = full_catalog_df.loc[:, 'pred_Z'].mask(filter_pred_z, full_catalog_df.loc[filter_pred_z, 'Label'])
    full_catalog_df = full_catalog_df.drop(columns=['Label'])
    full_catalog_df.loc[:, 'pred_Z'] = np.around(full_catalog_df.loc[:, 'pred_Z'], decimals=3)

In [20]:
full_catalog_df['pred_prob_class']    = (full_catalog_df.loc[:, 'Prob_AGN']   >= gv.cal_AGN_thresh).astype(int)
full_catalog_df['pred_prob_radio']    = (full_catalog_df.loc[:, 'Prob_radio'] >= gv.cal_radio_thresh).astype(int)

In [21]:
full_catalog_df['pred_radio_AGN']     = (np.array(full_catalog_df.loc[:, 'pred_class'] == 1) & np.array(full_catalog_df.loc[:, 'pred_radio'] == 1)).astype(int)
full_catalog_df['Score_rAGN']         = full_catalog_df.loc[:, 'Score_AGN'] * full_catalog_df.loc[:, 'Score_radio']
full_catalog_df['pred_prob_rAGN']     = (np.array(full_catalog_df.loc[:, 'pred_prob_class'] == 1) & np.array(full_catalog_df.loc[:, 'pred_prob_radio'] == 1)).astype(int)
full_catalog_df['Prob_rAGN']          = full_catalog_df.loc[:, 'Prob_AGN'] * full_catalog_df.loc[:, 'Prob_radio']

rad_score_scaler                      = MinMaxScaler()
full_catalog_df['scaled_score_radio'] = rad_score_scaler.fit_transform(full_catalog_df.loc[:, 'Score_radio'].values.reshape(-1, 1))
full_catalog_df['scaled_score_rAGN']  = full_catalog_df.loc[:, 'Score_AGN'] * full_catalog_df.loc[:, 'scaled_score_radio']

In [22]:
full_catalog_df.loc[:, ['class', 'pred_prob_class', 'LOFAR_detect', 'pred_prob_radio', 'Z', 'pred_Z']].describe()

,class,pred_prob_class,LOFAR_detect,pred_prob_radio,Z,pred_Z
count,118734.000000,1.513688e+07,1.513688e+07,1.513688e+07,134234.000000,1.513688e+07
mean,0.425641,6.623990e-01,1.239179e-02,9.470163e-02,0.876576,2.034261e+00
std,0.494442,4.728917e-01,1.106265e-01,2.928024e-01,0.779380,5.600509e-01
min,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,-0.004490,5.000000e-03
25%,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.381018,1.943000e+00
50%,0.000000,1.000000e+00,0.000000e+00,0.000000e+00,0.623500,2.129000e+00
75%,1.000000,1.000000e+00,0.000000e+00,0.000000e+00,1.282000,2.129000e+00
max,1.000000,1.000000e+00,1.000000e+00,1.000000e+00,7.028330,4.796000e+00


Obtain intermediate metrics

In [23]:
filter_radio_AGN_t      = np.array(full_catalog_df.loc[:, 'class'] == 1) & np.array(full_catalog_df.loc[:, 'LOFAR_detect'] == 1)
filter_gal_AGN_t        = np.array(full_catalog_df.loc[:, 'class'] == 0) | np.array(full_catalog_df.loc[:, 'class'] == 1)
total_size              = len(full_catalog_df)
filter_AGN_t            = np.array(full_catalog_df.loc[:, 'class'] == 1)
num_AGN_t               = np.sum(filter_AGN_t)
num_gal_t               = np.sum(np.array(full_catalog_df.loc[:, 'class'] == 0))
num_radio_t             = np.sum(np.array(full_catalog_df.loc[:, 'LOFAR_detect'] == 1))
num_radio_AGN_t         = np.sum(filter_radio_AGN_t)

In [24]:
# filter_radio_AGN_p      = np.array(full_catalog_df.loc[:, 'Score_AGN']          >= threshold_AGN) &\
#                             np.array(full_catalog_df.loc[:, 'Score_radio']      >= threshold_radio)
filter_radio_AGN_p      = np.array(full_catalog_df.loc[:, 'pred_prob_class']      == 1) &\
                            np.array(full_catalog_df.loc[:, 'pred_prob_radio']    == 1)
filt_hiz_rAGN_p         = filter_radio_AGN_p * np.array(full_catalog_df.loc[:, 'pred_Z'] >= gv.high_z_limit)
filter_AGN_p            = np.array(full_catalog_df.loc[:, 'pred_prob_class']      == 1)
filter_radio_p          = np.array(full_catalog_df.loc[:, 'pred_prob_radio']      == 1)
num_AGN_p               = np.sum(filter_AGN_p)
num_gal_p               = np.sum(np.array(full_catalog_df.loc[:, 'pred_prob_class'] == 0))
num_radio_p             = np.sum(filter_radio_p)
num_radio_AGN_p         = np.sum(filter_radio_AGN_p)

### Select sources predicted to be Radio AGN (optional)

In [25]:
# full_catalog_df         = full_catalog_df.loc[filter_radio_AGN_p]

In [26]:
full_catalog_df.loc[:, ['class', 'pred_prob_class', 'LOFAR_detect', 'pred_prob_radio', 'Z', 'pred_Z']].describe()

,class,pred_prob_class,LOFAR_detect,pred_prob_radio,Z,pred_Z
count,118734.000000,1.513688e+07,1.513688e+07,1.513688e+07,134234.000000,1.513688e+07
mean,0.425641,6.623990e-01,1.239179e-02,9.470163e-02,0.876576,2.034261e+00
std,0.494442,4.728917e-01,1.106265e-01,2.928024e-01,0.779380,5.600509e-01
min,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,-0.004490,5.000000e-03
25%,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.381018,1.943000e+00
50%,0.000000,1.000000e+00,0.000000e+00,0.000000e+00,0.623500,2.129000e+00
75%,1.000000,1.000000e+00,0.000000e+00,0.000000e+00,1.282000,2.129000e+00
max,1.000000,1.000000e+00,1.000000e+00,1.000000e+00,7.028330,4.796000e+00


Add individual metrics for redshift

In [27]:
full_catalog_df['Delta_z_N'] = np.around((full_catalog_df.loc[:, 'pred_Z'] - full_catalog_df.loc[:, 'Z']) /\
                            (1 + full_catalog_df.loc[:, 'Z']), decimals=3)

full_catalog_df['sigma_NMAD'] = np.around(1.48 * np.abs(full_catalog_df.loc[:, 'pred_Z'] - full_catalog_df.loc[:, 'Z']) /\
                            (1 + full_catalog_df.loc[:, 'Z']), decimals=3)

Numerical summary.

In [28]:
str_0_t   = f'Out of {total_size:,} initial sources in {used_area},\n'
str_1_t   = f'{num_gal_t:,} are confirmed to be galaxies. On the other side,\n'
str_2_t   = f'{num_AGN_t:,} are confirmed to be AGN. And, from the AGN,\n'
str_3_t   = f'{num_radio_AGN_t:,} are detected in radio.'

str_0_p   = f'Out of {num_radio_AGN_t:,} initial radio-detected AGN in {used_area},\n'
str_1_p   = f'{num_gal_p:,} are predicted to be galaxies. On the other side,\n'
str_2_p   = f'{num_AGN_p:,} are predicted to be AGN. And, from the predicted AGN,\n'
str_3_p   = f'{num_radio_AGN_p:,} are predicted to be detected in radio.'

In [29]:
print('-' * 60)
print(str_0_t + str_1_t + str_2_t + str_3_t)
print('-' * 60)
print(str_0_p + str_1_p + str_2_p + str_3_p)
print('-' * 60)

------------------------------------------------------------
Out of 15,136,878 initial sources in HETDEX,
68,196 are confirmed to be galaxies. On the other side,
50,538 are confirmed to be AGN. And, from the AGN,
6,419 are detected in radio.
------------------------------------------------------------
Out of 6,419 initial radio-detected AGN in HETDEX,
5,110,225 are predicted to be galaxies. On the other side,
10,026,653 are predicted to be AGN. And, from the predicted AGN,
79,847 are predicted to be detected in radio.
------------------------------------------------------------


In [30]:
cols_4_table = show_columns = ['Name', 'RA_ICRS', 'DE_ICRS', 'TYPE', 'band_num', 'class', 'pred_class',\
                               'pred_prob_class', 'Score_AGN', 'Prob_AGN', 'LOFAR_detect', 'pred_radio',\
                               'pred_prob_radio', 'Score_radio', 'Prob_radio', 'radio_AGN', 'pred_radio_AGN',\
                               'pred_prob_rAGN', 'Score_rAGN', 'Prob_rAGN', 'Z', 'pred_Z']

In [31]:
cols_4_export_S82    = ['Fint_VLAS82']
cols_4_export_HETDEX = ['Sint_LOFAR']
cols_4_export_COSMOS = ['Flux_COSMOSVLA3']

cols_4_exp_all       = {'S82': cols_4_export_S82, 'HETDEX': cols_4_export_HETDEX, 'COSMOS': cols_4_export_COSMOS}

cols_photo           = ['W1mproPM', 'W2mproPM', 'gmag', 'rmag', 'imag', 'zmag', 
                        'ymag', 'Jmag', 'Hmag', 'Kmag', 'W3mag', 'W4mag']

cols_4_export        = cols_4_table + cols_4_exp_all[used_area] + cols_photo

In [32]:
filter_new_rAGN = full_catalog_df.loc[:, 'radio_AGN'] == 0
if predict_only_hi_z:
    filter_high_z   = full_catalog_df.loc[:, 'pred_Z']    >= gv.high_z_limit
elif not predict_only_hi_z:
    filter_high_z   = np.ones_like(full_catalog_df.loc[:, 'pred_Z']).astype(bool)

In [33]:
full_catalog_df.loc[:, 'TYPE'] = full_catalog_df.loc[:, 'TYPE'].mask(full_catalog_df.loc[:, 'TYPE'] == '    ', 'CCCC')

In [34]:
display(full_catalog_df.loc[filter_high_z, cols_4_export].sort_values(by=['pred_Z'], ascending=False).head(15))

,Name,RA_ICRS,DE_ICRS,TYPE,band_num,class,pred_class,pred_prob_class,Score_AGN,Prob_AGN,LOFAR_detect,pred_radio,pred_prob_radio,Score_radio,Prob_radio,radio_AGN,pred_radio_AGN,pred_prob_rAGN,Score_rAGN,Prob_rAGN,Z,pred_Z,Sint_LOFAR,W1mproPM,W2mproPM,gmag,rmag,imag,zmag,ymag,Jmag,Hmag,Kmag,W3mag,W4mag
7355795,J133422.63+475033.4,203.594269,47.842628,QX,8,1.0,1,1,0.500056,0.892185,1,0,0,0.048814,0.062214,1,0,0,0.024410,0.055506,4.958,4.796,14.569016,18.983999,19.130001,23.299999,20.933100,19.620399,19.306101,19.200300,17.450001,17.24,16.59,16.67,14.62
3226938,J111714.13+540919.4,169.308868,54.155396,Q,8,1.0,1,1,0.500068,0.926110,1,0,1,0.208025,0.225272,1,0,1,0.104026,0.208627,4.700,4.748,16.954622,19.456999,19.754002,23.299999,21.331800,20.118099,19.834101,19.797501,17.450001,17.24,16.59,16.67,14.62
4264628,J141000.75+460134.5,212.503098,46.026260,,6,NaN,0,0,0.499961,0.205264,0,0,1,0.248813,0.261320,0,0,0,0.124397,0.053640,NaN,4.715,17.520000,19.681999,19.809999,23.299999,20.781799,19.471001,19.164101,19.027300,17.450001,17.24,16.59,16.67,14.62
9544254,J132514.21+534447.1,201.309235,53.746429,,6,NaN,0,1,0.500007,0.578804,0,0,1,0.351672,0.345250,0,0,1,0.175838,0.199832,NaN,4.711,17.520000,18.823999,19.125999,23.299999,20.935801,19.635599,19.379000,19.478600,17.450001,17.24,16.59,16.67,14.62
10831188,J124400.05+553406.8,191.000183,55.568558,Q,8,1.0,1,1,0.500053,0.880835,0,0,1,0.736330,0.616554,0,0,1,0.368204,0.543082,4.623,4.654,17.520000,18.931999,19.226000,23.299999,21.077101,19.595501,19.544100,19.271700,17.450001,17.24,16.59,16.67,14.62
13292816,J143508.80+485521.4,218.786652,48.922630,Q,8,1.0,1,1,0.500043,0.833508,0,0,0,0.131533,0.152147,0,0,0,0.065772,0.126816,4.776,4.652,17.520000,19.424000,19.650002,23.299999,21.011400,19.706301,19.509300,19.394300,17.450001,17.24,16.59,16.67,14.62
12355845,J144321.14+501908.4,220.838120,50.319016,,5,NaN,0,1,0.500007,0.578804,0,0,1,0.937123,0.794128,0,0,1,0.468568,0.459644,NaN,4.606,17.520000,19.511999,19.809999,23.299999,21.092501,19.449699,19.490801,21.400000,17.450001,17.24,16.59,16.67,14.62
13669337,J143442.05+532102.5,218.675201,53.350716,,7,NaN,0,0,0.499916,0.046781,0,0,1,0.438486,0.410208,0,0,0,0.219206,0.019190,NaN,4.594,17.520000,19.473999,19.809999,23.299999,20.897600,20.666800,20.399200,21.400000,17.450001,17.24,16.59,16.67,14.62
3108302,J111513.19+532105.5,168.804932,53.351532,,7,NaN,0,0,0.499973,0.282668,0,0,0,0.089812,0.108494,0,0,0,0.044904,0.030668,NaN,4.586,17.520000,19.511000,19.809999,23.299999,20.940901,20.626600,20.388500,21.400000,17.450001,17.24,16.59,16.67,14.62
5398215,J140947.84+514530.7,212.449341,51.758556,,5,NaN,0,0,0.499945,0.123657,0,0,1,0.305554,0.308716,0,0,0,0.152760,0.038175,NaN,4.584,17.520000,19.552999,19.809999,23.299999,21.259800,20.958599,20.683599,21.400000,17.450001,17.24,16.59,16.67,14.62


In [35]:
if save_preds_flag:
    full_catalog_df.loc[filter_high_z, cols_4_export].sort_values(by=['pred_Z'],
     ascending=False).to_csv(gv.preds_path + f'predicted_rAGN_{used_area}.csv', index_label='ID')

---

In [36]:
temp_flag = True

In [37]:
if temp_flag:
    cols_AGN   = ['g_r', 'r_i', 'r_J', 'i_z', 'i_y', 
                  'z_y', 'z_W2', 'y_J', 'y_W1', 'y_W2', 'J_H', 'H_K', 
                  'H_W3', 'W1_W2', 'W1_W3', 'W3_W4']  # Only colours (no 'band_num', 'W4mag')
    cols_radio = ['g_r', 'g_i', 'r_i', 'r_z', 'i_z', 
                  'z_y', 'z_W1', 'y_J', 'y_W1', 'J_H', 'H_K', 'K_W3', 
                  'K_W4', 'W1_W2', 'W2_W3']  # Only colours (no 'band_num', 'W4mag')
    cols_z     = ['g_r', 'g_W3', 'r_i', 'r_z', 'i_z', 
                  'i_y', 'z_y', 'y_J', 'y_W1', 'J_H', 'H_K', 'K_W3', 
                  'K_W4', 'W1_W2', 'W2_W3']  # Only colours (no 'band_num', 'W4mag')
    
    cols_cols = list(np.unique(cols_AGN + cols_radio + cols_z))

In [38]:
if temp_flag:
    
    add_columns  = ['band_num', 'class', 'pred_prob_class', 'Score_AGN', 'Prob_AGN', 
                    'LOFAR_detect',  'pred_prob_radio', 'Score_radio', 'Prob_radio', 
                    'radio_AGN', 'pred_prob_rAGN', 'Score_rAGN', 'Prob_rAGN', 'Z', 'pred_Z']
    used_colours = cols_cols

In [39]:
if temp_flag:
    saving_data = full_catalog_df.loc[:, add_columns + cols_photo + used_colours]
    saving_data['ID'] = saving_data.index
    file_name         = gv.preds_path + f'{used_area}_for_prediction.h5'
    file_name_pqt     = gv.preds_path + f'{used_area}_for_prediction.parquet'
    saving_data.to_hdf(file_name, key='df')
    saving_data.to_parquet(file_name_pqt, index=True, engine='fastparquet')
    print(f'Data from {used_area} saved in files {file_name} and {file_name_pqt}')

Data from HETDEX saved in files pred_rAGN/HETDEX_for_prediction.h5 and pred_rAGN/HETDEX_for_prediction.parquet
